# 如何实现自定义优化算法



In [ ]:
import torch
from torch.optim import Optimizer

from sklearn import datasets
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_boston(ratio=0.8):
    X, Y = datasets.load_boston(True)
    Y.shape = -1, 1
    
    # normalization
    X = X/80
    Y = Y/(np.max(Y) - np.min(Y))
    
    num_samples = len(Y)
    num_train = math.ceil(num_samples * ratio)
    
    # 随机打乱数据
    idx = np.random.permutation(np.arange(num_samples))
    traindata = X[idx[:num_train]], Y[idx[:num_train]]
    validdata = X[idx[num_train:]], Y[idx[num_train:]]
    
    return traindata, validdata

利用pytorch下的`torch.optim.Optimizer`来实现一个自己的优化算法

Reference: https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer

In [ ]:
class GradientDescent(torch.optim.Optimizer):
    def __init__(self, params, lr=1e-3):
        # 将参数导回Optimizer，这样可以允许lr_scheduler修改lr
        defaults = dict(lr=lr)
        super().__init__(params, defaults)
        
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data

                p.data.add_(-group['lr'], grad) # in-place add

测试

In [ ]:
(X_train, Y_train), (X_valid, Y_valid) = load_boston()
X_train, Y_train = torch.Tensor(X_train), torch.Tensor(Y_train)
X_valid, Y_valid = torch.Tensor(X_valid), torch.Tensor(Y_valid)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(13, 50),
    torch.nn.Sigmoid(),
    torch.nn.Linear(50, 1)
)

In [ ]:
loss = torch.nn.MSELoss()

In [ ]:
# 使用自定义优化器
opt = GradientDescent(model.parameters(), lr=1e-5)

In [ ]:
valid_losses = []
train_losses = []
for i in range(1000):
    X, Y = X_train, Y_train
    
    opt.zero_grad()
    l = loss(model(X), Y)
    l.backward()
    opt.step()
    
    with torch.no_grad():
        cur_train_loss = l
        cur_valid_loss = loss(Y_valid, model(X_valid))
        valid_losses.append(cur_valid_loss) 
        train_losses.append(cur_train_loss)

        if i%100 == 0:
            print(f"Iter {i}: loss {cur_train_loss:.4f}, valid loss {cur_valid_loss:.4f}")

In [ ]:
plt.plot(train_losses)
plt.plot(valid_losses)
plt.legend(["train loss", "validation loss"])

作业五：

借助本文档的代码结构，实现带动量的随机梯度下降算法(SGD with momentum) （截止时间：12.16）

参考材料：https://blog.csdn.net/bvl10101111/article/details/72615621